In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from scipy import optimize
from numba import njit

### Wilson-Cowan equations

In the general case, the Wilson-Cowan equations have the form,

\begin{align}
\begin{split}
    \tau_E \frac{dE}{dt} &= -E + S(\omega_{EE} E - \omega_{EI} I + P_E) \\
    \tau_I \frac{dI}{dt} &= -I + S(\omega_{IE} E - \omega_{II} I + P_I) \\
\end{split}
\end{align}

In [ ]:

@njit
def S(x):
    """
    S function
    """
    return np.tanh(x/2)/2

#Define S prime
@njit
def S_prime(x):
    return 1/4*(1-np.tanh(x/2)**2)

#Define the System of equations
@njit
def wilson_covan (y, wEE, wEI, wIE, wII, P, t):
    E, I = y
    P_E, P_I = P
    tau_E, tau_I = t
    dEdt = (-E + S (wEE * E - wEI * I + P_E)) / tau_E
    dIdt = (-I + S (wIE * E - wII * I + P_I)) / tau_I
    return np.array([dEdt, dIdt])

#Define the Jacobian matrix 
@njit
def jacobian(y, wEE, wEI, wIE, wII, P, t):
    E, I = y
    P_E, P_I = P
    tau_E, tau_I = t

    dEdE = (-1 + wEE * S_prime(wEE * E - wEI * I + P_E))/tau_E
    dEdI = -wEI * S_prime(wEE * E - wEI * I +P_E) / tau_E
    dIdE = wIE * S_prime(wIE * E - wII * I + P_I) / tau_I
    dIdI = (-1 - wII*S_prime(wIE * E - wII * I + P_I))/ tau_I

    return np.array([dEdE, dEdI, dIdE, dIdI])


### Task 1: Dynamical Regimes

In the first task we are interested in analyzing the different dynamical regimes that the model produce. The ones that we will consider are,

+ Quiescent: produced in the cases where there